# 1. 모델 이해하기

##### 1번 레이어 : 합성곱층(Convolution layer)
합성곱(in_channel=1, out_channerl=32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU

맥스풀링(kernel_size=2, stride=2)

##### 2번 레이어 : 합성곱 층 (Convolutional layer)
합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU

맥스 풀링(kernel_size=2, stride=2)

##### 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size x 7 x 7 -> batch_size x 3136

전결합층(뉴런 10개) + 활성화 함수 Softmax

# 2. 모델 구현하기

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs=torch.Tensor(1,1,28,28) # 배치 크기 x 채널 x 높이 x 너비 크기의 텐서를 선언
print('텐서의 크기 : {}'.format(inputs.shape))

텐서의 크기 : torch.Size([1, 1, 28, 28])


In [3]:
conv1=nn.Conv2d(1,32,3,padding=1) # 1채널입력 32채널추출 커널사이즈3 패딩1
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [4]:
conv2=nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
pool=nn.MaxPool2d(2) # 커널사이즈2 스트라이드2
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [6]:
out=conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [7]:
out=pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [8]:
out=conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [9]:
out=pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [10]:
out.size(0) # .size(n)은 n번째 차원에 접근하게 해줌

1

In [11]:
out.size(1)

64

In [12]:
out.size(2)

7

In [13]:
out.size(3)

7

In [14]:
# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라.
out=out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


In [15]:
fc=nn.Linear(3136, 10) #input_dim, output_dim -> 전결합층(Fully Connected layer)
out=fc(out)
print(out.shape)

torch.Size([1, 10])


# 3. CNN으로 MNIST 분류하기

In [5]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [6]:
device='cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPY 사용 가능이면 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
learning_rate=0.001
training_epochs=15
batch_size=100

In [8]:
mnist_train=dsets.MNIST(root='MNIST_data/',
                        train=True,
                        transform=transforms.ToTensor(),
                        download=True)

mnist_test=dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [9]:
data_loader=torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [11]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.layer1=torch.nn.Sequential(
            torch.nn.Conv2d(1,32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer2=torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=torch.nn.Linear(7*7*64, 10, bias=True)

        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        return out

In [12]:
model=CNN().to(device)

In [13]:
criterion=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
total_batch=len(data_loader)
print('총 배치의 수: {}'.format(total_batch))

총 배치의 수: 600


In [16]:
for epoch in range(training_epochs):
    avg_cost=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        hypothesis=model(X)
        cost=criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch+1, avg_cost))

KeyboardInterrupt: 

In [15]:
with torch.no_grad():
    X_test=mnist_test.test_data.view(len(mnist_test), 1,28, 28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)

    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction, 1)==Y_test
    accuracy=correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.08299999684095383


In [2]:
import torch